In [15]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [16]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='cali_housing', task='regression', batch_size=32)

xtrain size 16512
xtest size 4128
y_train size 16512
y_test size 4128


In [17]:
output_size

1

In [18]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [19]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='regression',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [20]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [21]:
output_directory = './output.csv'
run_experiment(DTEmlp4b, trails=50, output_directory=output_directory)

Trail 0: 

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: regression
Randomly initializing 4 models
Model 1:


Training Process: 100%|██████████| 10/10 [00:15<00:00,  1.55s/it]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:16<00:00,  1.68s/it]


./saved_models/model_2.pth
Model 3:


...

Training Process: 100%|██████████| 10/10 [00:30<00:00,  3.07s/it]


./saved_models/model_8.pth
Model 6:


Training Process: 100%|██████████| 10/10 [00:29<00:00,  2.95s/it]


./saved_models/model_6.pth
 Minimum RMSE Obtained over all training: 1.0040540402249774
|| TRAINING SINGLE MODEL || 
Task: Regression


Training Process: 100%|██████████| 70/70 [03:32<00:00,  3.03s/it]


In [23]:
df = pd.read_csv(output_directory)
df.head()

,DTE_rmse,DTE_test,DTE_train,single_rmse,single_test,single_train
0,1.011779,1.057935,1.071236,0.994219,1.034895,1.125871
1,1.025821,1.086470,1.064970,1.009424,1.065298,1.126967
2,1.022039,1.079341,1.073850,1.059684,1.188070,1.180093
3,1.006106,1.049031,1.084698,1.005596,1.087355,1.254352
4,0.949788,0.931172,1.060259,1.012790,1.090921,1.215553


In [24]:
print(f'single rmse mean: {df["single_rmse"].mean()}  DTE rmse mean: {df["DTE_rmse"].mean()}')
print(f'single rmse var: {df["single_rmse"].var()}  DTE rmse var: {df["DTE_rmse"].var()}')
print(f'single loss mean: {df["single_test"].mean()}  DTE loss mean: {df["DTE_test"].mean()}')
print(f'single loss var: {df["single_test"].var()}  DTE loss var: {df["DTE_test"].var()}')

single rmse mean: 1.0123970238085414  DTE rmse mean: 0.9851284931577504
single rmse var: 0.0004490685175451768  DTE rmse var: 0.0005479001671928898
single loss mean: 1.0849927160617412  DTE loss mean: 1.0038443388874712
single loss var: 0.002488589810680656  DTE loss var: 0.002231496839095565


: 

In [ ]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [ ]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)